<a href="https://colab.research.google.com/github/E-CG/AI4ENG/blob/master/03%20-%20Modelos-e-iteraciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **📦 Librerias y paquetes para la ejecución del notebook.**

In [ ]:
! pip install py7zr

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle
# Aquí debes cambiar la dirección donde tengas tus credenciales de Kaggle
! cp /content/drive/MyDrive/Modelos_I/credentials_kaggle/kaggle.json ~/.kaggle/kaggle.json

In [5]:
! kaggle competitions download favorita-grocery-sales-forecasting
! unzip favorita-grocery-sales-forecasting.zip

favorita-grocery-sales-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  favorita-grocery-sales-forecasting.zip
replace holidays_events.csv.7z? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
# Librerias uso básico
import numpy as np
import pandas as pd
import math as m
import time
import py7zr
import os
from subprocess import check_output

# Librerias preprocesado
from mlxtend.preprocessing import minmax_scaling
import datetime as dt

# Librerias para gráficar
import seaborn as sns
import matplotlib.pyplot as plt

# Funciones de sklearn
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression,ElasticNet,Ridge,Lasso
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error

## **🗂️Leyendo y extrayendo los archivos .csv**

In [7]:
# Ruta al archivo 7z en Google Drive
sevenzip_file_path = '/content'

# Directorio de destino para la extracción
extracted_dir = '/content/extracted_data/'

# Crear el directorio de destino si no existe
os.makedirs(extracted_dir, exist_ok=True)

# Listar los archivos .7z en el directorio de entrada
files_to_extract = [file for file in os.listdir(sevenzip_file_path) if file.endswith('.7z')]

# Iterar a través de los archivos y descomprimirlos
for file_to_extract in files_to_extract:
    with py7zr.SevenZipFile(os.path.join(sevenzip_file_path, file_to_extract), mode='r') as z:
        z.extractall(path=extracted_dir)

In [8]:
stores = pd.read_csv('/content/extracted_data/stores.csv')
items = pd.read_csv('/content/extracted_data/items.csv')
holidays_e = pd.read_csv('/content/extracted_data/holidays_events.csv', parse_dates=["date"])
transactions = pd.read_csv('/content/extracted_data/transactions.csv', parse_dates=["date"])
oil = pd.read_csv('/content/extracted_data/oil.csv', parse_dates=["date"])

# Cargar el archivo de entrenamiento en chunks
chunked_dfs = pd.read_csv("/content/extracted_data/train.csv",
                          chunksize=20000,
                          usecols=[0,1,2,3,4,5],
                          parse_dates=['date'],
                          low_memory=False)

print('Archivos cargados 🗞️✅')

Archivos cargados 🗞️✅


In [9]:
# Agrupar por estado y obtener el primer store_nbr en cada estado
tienda_por_estado = stores.groupby('state')['store_nbr'].first()
tienda_por_estado

state
Azuay                             37
Bolivar                           19
Chimborazo                        14
Cotopaxi                          12
El Oro                            40
Esmeraldas                        43
Guayas                            24
Imbabura                          15
Loja                              38
Los Rios                          31
Manabi                            52
Pastaza                           22
Pichincha                          1
Santa Elena                       25
Santo Domingo de los Tsachilas     5
Tungurahua                        23
Name: store_nbr, dtype: int64

In [10]:
# Inicializar una lista para almacenar los DataFrames filtrados
datos_filtrados = []

# Iterar a través de los chunks
for chunk in chunked_dfs:
    # Crear una máscara booleana para las tiendas seleccionadas
    mask_tiendas = chunk['store_nbr'].isin(stores['store_nbr'])
    # Crear una máscara booleana para las fechas entre 2016-01-01 a 2016-12-31
    mask_fechas = (chunk['date'] >= '2016-01-01') & (chunk['date'] <= '2016-12-31')
    # Aplicar ambas máscaras
    mask = mask_tiendas & mask_fechas

    # Aplicar la máscara y agregar las filas filtradas a la lista
    trozo_filtrado = chunk[mask]
    datos_filtrados.append(trozo_filtrado)

# Concatenar los DataFrames filtrados en uno solo
train_store_state = pd.concat(datos_filtrados, ignore_index=True)

In [11]:
train_store_state.head(3)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,66458908,2016-01-01,25,105574,12.0,False
1,66458909,2016-01-01,25,105575,9.0,False
2,66458910,2016-01-01,25,105857,3.0,False


In [12]:
# Seleccionar aleatoriamente 70.000 filas de 'train_store_state'
train_sample = train_store_state.sample(n=70000)

In [13]:
train_sample.head(3)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
31815211,98274119,2016-11-28,31,1458847,1.0,False
27505972,93964880,2016-10-15,39,1920284,1.0,False
2199601,68658509,2016-01-25,16,977007,2.0,False


## **📄Juntando los demás archivos en train.csv y generando dataframes de prueba y entrenamiento**

Haré uso de un pipeline utilizando dos clases, una para unir los demás archivos teniendo como base el dataframe de entrenamiento 'train_store_state'

In [14]:
class MergeDataTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        # Constructor de la clase
        print("Inicializando MergeDataTransformer")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Unir DataFrames
        train_stores = X[0].merge(X[1], on='store_nbr')
        train_stores_oil = train_stores.merge(X[2], on='date')
        train_stores_oil_items = train_stores_oil.merge(X[3], on='item_nbr')
        train_stores_oil_items_transactions = train_stores_oil_items.merge(X[4], on=['date', 'store_nbr'])
        train_stores_oil_items_transactions_hol = train_stores_oil_items_transactions.merge(X[5], on='date')

        # Copiar DataFrame
        data = train_stores_oil_items_transactions_hol.copy(deep=True)

        # Cambiar booleanos a enteros
        data['onpromotion'] = data['onpromotion'].astype(int)
        data['transferred'] = data['transferred'].astype(int)

        # Cambiar nombres de columnas
        data.rename(columns={'type_x': 'st_type', 'type_y': 'hol_type'}, inplace=True)

        # Eliminar la columna 'id'
        data.drop(['id'], axis=1, inplace=True)

        # Mostrar las primeras filas del DataFrame resultante
        print(data.head())

        # Manipular la columna de fecha
        data['date'] = pd.to_datetime(data['date'])
        data['date'] = data['date'].map(dt.datetime.toordinal)

        return data

Divisor de Datos (SplitDataTransformer) ➗

Este transformador personalizado elimina la columna 'date' del conjunto de datos original y divide los datos en tres partes:

1. **Datos Numéricos (`data_numeric_df`):** Incluye todas las columnas numéricas del conjunto de datos original.

2. **Datos Categóricos (`data_categorical_df`):** Incluye todas las columnas categóricas del conjunto de datos original.

3. **Fecha (`data_date_df`):** Incluye la columna 'date' original.

In [15]:
class SplitDataTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        # Constructor de la clase
        print("Inicializando SplitDataTransformer")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Eliminar la columna 'date' del DataFrame original
        df_without_date = X.drop(['date'], axis=1)

        # Obtener columnas numéricas y categóricas
        all_columns = df_without_date.columns
        numeric_columns = df_without_date._get_numeric_data().columns
        categorical_columns = list(set(all_columns) - set(numeric_columns))

        # Crear DataFrames separados para datos numéricos, categóricos y de fecha
        data_numeric_df = X[numeric_columns]
        data_categorical_df = X[categorical_columns]
        data_date_df = X['date']

        return data_numeric_df, data_categorical_df, data_date_df

Procesamiento de Datos (ProcessData) 🛤️

Este transformador personalizado realiza el procesamiento de datos, incluyendo:

1. **Datos Numéricos (`data_num_df`):**
   - Imputa valores nulos en atributos numéricos utilizando la estrategia de la media.
   - Aplica escalado estándar a los datos numéricos.

2. **Datos Categóricos (`data_cat_df`):**
   - Utiliza un codificador one-hot para representar las variables categóricas.
   - Convierte la representación codificada a un DataFrame.

3. **Fecha (`X[2]`):**
   - Mantiene la columna de fechas sin cambios.

In [16]:
class ProcessData(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("Inicializando ProcessData")

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Datos numéricos
        # Imputar valores nulos en atributos numéricos
        imputer = SimpleImputer(strategy="mean", copy=True)
        num_imputed = imputer.fit_transform(X[0])
        data_num_df = pd.DataFrame(num_imputed, columns=X[0].columns, index=X[0].index)

        # Aplicar escalado estándar
        scaler = StandardScaler()
        num_scaled = scaler.fit_transform(data_num_df)
        data_num_df = pd.DataFrame(num_scaled, columns=X[0].columns, index=X[0].index)

        # Datos categóricos
        # One-hot encoder
        cat_encoder = OneHotEncoder(sparse=False)
        data_cat_1hot = cat_encoder.fit_transform(X[1])

        # Convertir a DataFrame con n*99, donde n es el número de filas y 99 es el número de categorías
        data_cat_df = pd.DataFrame(data_cat_1hot, columns=cat_encoder.get_feature_names_out(), index=X[1].index)

        return data_num_df, data_cat_df, X[2]

Unión de DataFrames (JoinDataFrames) 🖇️

Este transformador es útil para combinar información de diferentes conjuntos de datos antes de proceder con el análisis o modelado posterior.

In [17]:
class JoinDataFrames(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("Inicializando JoinDataFrames")

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Unir DataFrames numéricos
        data_df = X[0].join(X[1])
        data_df = data_df.join(X[2])

        return data_df

Pipeline de Procesamiento de Datos 🔩

1. **Preparar Datos (`MergeDataTransformer`):**
   - Realiza las primeras manipulaciones de los datos.

2. **Dividir Datos (`SplitDataTransformer`):**
   - Separa los datos en categorías numéricas, categóricas y de fecha.

3. **Procesar Datos (`ProcessData`):**
   - Imputa valores nulos y aplica escalado a datos numéricos.
   - Codifica variables categóricas utilizando one-hot encoding.

4. **Unir Datos (`JoinDataFrames`):**
   - Combina DataFrames numéricos en uno solo.


In [18]:
# Definir la pipeline
pipeline = Pipeline([
    ('merge_data', MergeDataTransformer()),
    ('split_data', SplitDataTransformer()),
    ('process_data', ProcessData()),
    ('join_data', JoinDataFrames())
])

Inicializando MergeDataTransformer
Inicializando SplitDataTransformer
Inicializando ProcessData
Inicializando JoinDataFrames


Aplicar Pipeline y Dividir Datos ✅

In [19]:
# Aplicar la pipeline para transformar los datos
data_df = pipeline.fit_transform([train_sample, stores, oil, items, transactions, holidays_e])

# Dividir los datos según características
X = data_df.drop(['unit_sales', 'transactions'], axis=1)
Y = data_df[['unit_sales', 'transactions']]

        date  store_nbr  item_nbr  unit_sales  onpromotion      city  \
0 2016-11-28         31   1458847         1.0            0  Babahoyo   
1 2016-11-28         31   1085686         5.0            0  Babahoyo   
2 2016-11-28         31   1932492         1.0            0  Babahoyo   
3 2016-11-28         31    557257         1.0            1  Babahoyo   
4 2016-11-28         39   2027827         1.0            0    Cuenca   

      state st_type  cluster  dcoilwtico     family  class  perishable  \
0  Los Rios       B       10       45.66      DAIRY   2166           1   
1  Los Rios       B       10       45.66   CLEANING   3040           0   
2  Los Rios       B       10       45.66  BEVERAGES   1122           0   
3  Los Rios       B       10       45.66       EGGS   2502           1   
4     Azuay       B        6       45.66      DAIRY   2166           1   

   transactions hol_type    locale locale_name   description  transferred  
0          1188    Event  National     Ecuador

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [20]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [58]:
y_train.head(8)

,unit_sales,transactions
7181,0.385960,-0.616539
9207,-0.129510,1.014860
3828,-0.276787,-0.021977
1168,-0.276787,0.002645
7261,-0.055871,-0.279134
9665,-0.203148,1.915822
10468,-0.166329,0.632771
5832,-0.276787,0.055535


## **🔎Utilizando algunos modelos para hacer las primeras predicciones**

Función que evalua el rendimiento utilizando los estadisticos (MAE, MSE) tanto para los datos de entrenamiento como para los de prueba

In [21]:
def checkModelPerformance(model):
    model.fit(x_train.values, y_train.values)

    pred = model.predict(x_test.values)

    print("Media cuadrada del error: ", np.sqrt(mean_squared_error(y_test.values, pred)))
    print("Media absoluta del error: ", np.sqrt(mean_absolute_error(y_test.values, pred)))

In [22]:
print("LinearRegression")
checkModelPerformance(LinearRegression())

LinearRegression
Media cuadrada del error:  1026788.2252218189
Media absoluta del error:  157.87691737257336


In [23]:
print("Lasso")
checkModelPerformance(Lasso(alpha=0.1))

Lasso
Media cuadrada del error:  0.6401247774163764
Media absoluta del error:  0.6224353517781456


In [24]:
print("ElasticNet")
checkModelPerformance(ElasticNet())

ElasticNet
Media cuadrada del error:  0.8103722383719023
Media absoluta del error:  0.7187680751386157


In [25]:
print("Ridge")
checkModelPerformance(Ridge(alpha=1.0))

Ridge
Media cuadrada del error:  0.5274963790387254
Media absoluta del error:  0.5553622672267451


In [26]:
print("Random Forest")
checkModelPerformance(RandomForestRegressor(random_state=42))

Random Forest
Media cuadrada del error:  0.46603143568746175
Media absoluta del error:  0.4191897183858527


👀 Sé que en el anterior notebook el modelo que era mejor en este proyecto era ElasticNet pero con las mejoras aplicadas a los algoritmos lo es RandomForest que tiene la MAE menor.

## **🧾Metrica de evaluación**

Los pesos, $w_i$, se pueden encontrar en el archivo items.csv (consultar la página de Datos). Los artículos perecederos tienen un peso de 1.25, mientras que todos los demás artículos tienen un peso de 1.00.

Quiero utilizar la métrica de evaluación para saber cual es el rendimiento de los diferentes modelos que estoy probando.

In [27]:
def calculate_nwrmsle(y_true, y_pred, weights_dict):
    # y_pred es un 2D array quiero seleccionar la primera columna
    y_pred_log = np.log1p(y_pred[:, 0])

    # Aplicar logaritmo natural a los valores reales
    y_true_log = np.log1p(y_true)

    # Calcular el error cuadrático medio ponderado
    squared_errors = (y_true_log - y_pred_log) ** 2

    # Utilizar solo la columna de 'unit_sales' para weights_dict
    weighted_squared_errors = [squared_errors[i] * weights_dict[item_nbr] for i, item_nbr in enumerate(weights_dict)]

    # Calcular la métrica NWRMSLE
    nwrmsle = np.sqrt(np.sum(weighted_squared_errors) / len(y_true))

    return nwrmsle

In [28]:
# Lista de modelos
modelos = [
    LinearRegression(),
    Lasso(alpha=0.1),
    ElasticNet(),
    Ridge(alpha=1.0),
    RandomForestRegressor(random_state=42)
]

# Entrenar y evaluar cada modelo
for modelo in modelos:
    # Entrenar el modelo
    modelo.fit(x_train, y_train)

    # Hacer predicciones
    y_pred = modelo.predict(x_test.values)

    # Calcular NWRMSLE
    weights_dict = {item_nbr: 1.25 if perishable == 1 else 1.00 for item_nbr, perishable in zip(x_test['item_nbr'], x_test['perishable'])}
    nwrmsle = calculate_nwrmsle(y_test['unit_sales'].values, y_pred, weights_dict)

    # Imprimir nombre del modelo y NWRMSLE
    print(f"{type(modelo).__name__}: NWRMSLE = {nwrmsle}\n")

LinearRegression: NWRMSLE = nan



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
<ipython-input-27-4360e56cb060>:3: RuntimeWarning: invalid value encountered in log1p
  y_pred_log = np.log1p(y_pred[:, 0])
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but ElasticNet was fitted with feature names
  warnings.warn(


Lasso: NWRMSLE = 0.25297074571145894

ElasticNet: NWRMSLE = 0.25281192870060676

Ridge: NWRMSLE = 0.2634715835336895



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(


RandomForestRegressor: NWRMSLE = 0.2687959683202358



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


## **⚒️Refinando modelo**

### Optimización de Hiperparámetros con RandomForestRegressor ⚙️

En este código, se utiliza `RandomForestRegressor` junto con `GridSearchCV` para encontrar los mejores hiperparámetros para el modelo. `RandomForestRegressor` es un algoritmo que utiliza múltiples árboles de decisión para hacer predicciones.

La métrica utilizada para evaluar el rendimiento del modelo es el error cuadrático medio negativo (`neg_mean_squared_error`). La búsqueda en la cuadrícula se realiza mediante validación cruzada con 5 divisiones (`cv=5`).

In [29]:
# Crear un RandomForestRegressor con un estado aleatorio fijo para reproducibilidad
forest_reg = RandomForestRegressor(random_state=42)

# Definir los parámetros que deseas probar
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},  # Primer conjunto de hiperparámetros
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},  # Segundo conjunto de hiperparámetros
]

# Crear un objeto GridSearchCV
grid_search = GridSearchCV(
    forest_reg,  # El modelo a utilizar
    param_grid,  # La cuadrícula de parámetros a probar
    cv=5,  # Número de divisiones en la validación cruzada (k-fold)
    scoring='neg_mean_squared_error',  # Métrica a optimizar
    return_train_score=True  # Devolver puntajes de entrenamiento además de puntajes de prueba
)

# Entrenar el modelo con todas las combinaciones de hiperparámetros
grid_search.fit(x_train.values, y_train.values)

# Mostrar los resultados del GridSearchCV
print(grid_search)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')


In [30]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

In [31]:
grid_search.best_estimator_

RandomForestRegressor(max_features=8, n_estimators=30, random_state=42)

La búsqueda en la cuadrícula (`GridSearchCV`) se realiza sobre dos conjuntos de hiperparámetros diferentes. En el primer conjunto, se exploran combinaciones de `n_estimators` (número de árboles) y `max_features` (número máximo de características consideradas para dividir un nodo). En el segundo conjunto, se prueba desactivar el muestreo de bootstrap (`bootstrap: [False]`) junto con diferentes valores de `n_estimators` y `max_features`.


## **Iteraciones con el modelo**

In [48]:
productos = train_sample['item_nbr'].unique()
productos

array([1458847, 1920284,  977007, ..., 1986524, 1985099, 1449487])

In [ ]:
stores.city = stores.city.apply(lambda x: 1 if (x == 'Guayaquil' or x == 'Quito') else 0)
stores.drop(columns=['state', 'type', 'cluster'], inplace= True)

holidays_e.drop(columns = ['locale_name', 'description', 'transferred'], inplace = True)

In [69]:
class merge_data(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("merge_data -> init")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        pre_data = pd.merge(pd.merge(pd.merge(X, stores, on='store_nbr', how='left'), oil, on='date', how='left'), holidays_e, on='date', how='left')
        pre_data = pre_data.sort_values('date')

        pre_data.dcoilwtico = pre_data.dcoilwtico.fillna(method='bfill', axis=0).fillna(0)
        pre_data.type = pre_data.type.fillna('Normal')
        pre_data.locale = pre_data.locale.fillna('Normal')

        encoder = OneHotEncoder()
        categorical_cols = ['store_nbr', 'item_nbr', 'onpromotion', 'city', 'type', 'locale']

        encoder.fit(pre_data[categorical_cols])
        encoded_cols = encoder.transform(pre_data[categorical_cols]).toarray()
        encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(categorical_cols))

        pre_data = pd.concat([pre_data.drop(columns=categorical_cols), encoded_df], axis=1)
        data = pre_data.reset_index().drop(columns='index')

        return data

In [70]:
class split_data(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("split_data -> init")

    def fit(self, X, y=None):
        return self

    def transform(self, X):

        y = X.pop('unit_sales')
        X = X

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

        X_train = X_train.set_index('date')
        X_test = X_test.set_index('date')

        return X_train, X_test, y_train, y_test

In [ ]:
pipeline_prueba = Pipeline([
    ('merge_data', merge_data()),
    ('split_data', split_data())
])

In [ ]:
productos = train_sample['item_nbr'].unique()
lista = list()
for i in productos:

    try:

        X_train, X_test, y_train, y_test = pipeline_prueba.fit_transform(train_sample[train_sample.item_nbr == i])

        forest_reg.fit(X_train.values, y_train.values);

        prediction =  forest_reg.predict(X_train)

        lista.append([i, prediction[0]])

    except:
        lista.append([i, 0.0])

In [73]:
result = pd.DataFrame(lista, columns=['id', 'unit_sales'])
# Ordenar el DataFrame por la columna 'unit_sales' de forma descendente
result = result.sort_values('unit_sales', ascending=False)

result

,id,unit_sales
949,1464210,481.55000
1024,1473474,190.93573
516,507969,128.14979
1847,1584575,122.27000
2839,1392260,118.00000
...,...,...
3504,2011216,0.00000
3682,1391408,0.00000
1536,2045637,0.00000
3680,2045373,0.00000
